# Air Sac Tracking Prototype code
Lara S. Burchardt & Wim Pouw

lara.burchardt@donders.ru.nl

Next to body movements and acoustics, we would also like to track the air sac's inflation of the Siamangs. The air sac naturally forms a spherical(3D) or circular (2D) shape, and such shapes are retrievable from an image using the hough transform, and a bit of pre-processing of the images to get the optimal representation of the relevent edges of the air sac.

This code takes as input a sample video with a close up of a Siamang, and then tracks the air sac when it takes a sufficiently circular shape. The result is shown below; it is not perfect, but with a bit of smoothing this can function as a good air sac tracker. This code is very much under development, there are many ways to improve further.

In [20]:
# import the necessary packages
import numpy as np
import argparse
import cv2
import pandas as pd
from skimage import io, feature, color, measure, draw, img_as_float
import numpy as np
import csv
import os

#resused code from 
#https://pyimagesearch.com/2014/07/21/detecting-circles-images-using-opencv-hough-circles/
#https://stackoverflow.com/questions/31705355/how-to-detect-circlular-region-in-images-and-centre-it-with-python

## Circular Tracker

Using a video sample as input, the user first defines a region of interest in which to look for circular objects.
After a few pre-processing steps to increase tracking success, the Hough Transform is used to find circles. 
The radius and position of the circle is saved together with the used parameter combination used in pre-processing and tracking. 

# Code to track manually labeled image sequences

We can't use the ROI here, as very different images are in the sequence, so that the roi doesn't work.

Currently this doesn't work at all, only one example scene (comprised of multiple images) in batch 1 was tracked well

## Finding the best parameter combination

In this section we try different parameter combinations for pre-processing and tracking to find the parameter combination giving the highest correlation coefficient with the manually tracked results.

Parameters to differ:
- alpha (0.5;1;1.5;2)
- beta  (10,20,30,40,50)
- param2 (Hough Transform) (5;10;15;20)
- eroding iterations (first time: 6,7,8,9,10,11,12; second time: 2,3,4,5,6)
- canny parameters min 5,10,15,20,25,30 max 30,35,40,45,50,55,60

### To do:

1. combine 24 frames of manually tracked frames into video sequence (without the manuall track), 3 frames from all 6 videos?
2. define saving routine
3. define loops for different parameter combinations
4. calculate correlation coefficients for all combinations and manually tracked radii. (maybe in different program) 
5. choose 5 parameters spaces with best correlation coefficient, rerun with roi to see whether there is an increase


In [ ]:
testfolder = '../TestFrames/tracked_31082022/'
frames = os.listdir(testfolder)

column_names = ['frame','x','y', 'r', 'inputfile',                                    # actual results 
                'alpha', 'beta','canny_min', 'canny_max','erode_it_1','erode_it_2',   # values of brightness and contrast pre processing
                'param2']                                                             # parameters of hough circle transform 

# constants (define parameters for HoughTransform outside of function to be able to save and manipulate easier)
dp = 1
minDist = 10000
param1 = 10
# param2 is defined in loops
minRadius = 5
maxRadius = 250

# set up dataframe to save results 
global df
df = pd.DataFrame(columns = column_names)         
i=0

checkpoints = list(range(20,533736*20,100000))
# loops for different parameter combinations
for alpha in [0.5,1,1.5,2]:
#for alpha in [0.5]:
    for beta in [10,20,30,40,50,60]:
    #for beta in [10]:
        for canny_min in [5,10,15,20,25,30]:
        #for canny_min in [5]:
            for canny_max in [30,35,40,45,50,55,60]:
            #for canny_max in [30]:
                for erode_it_1 in [6,7,8,9,10,11,12]:
                #for erode_it_1 in [10]:
                    for erode_it_2 in [2,3,4,5,6]:
                    #for erode_it_2 in [2]:                     
                        for param2 in [5,10,15,20]:
                            # Opens the Video file
                            # Using cv2.imread() method
                            #initialize iterator
                            
                            for im in frames:
                                i=i+1
                                frame = cv2.imread(testfolder+im)
                                ############################detect circles   
                                output=frame.copy()
                                # transform to grayscale image only using the roi part of the image
                                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                                output_gray=gray.copy()
                                # increasing brightness and contrast
                                gray_light = cv2.convertScaleAbs(gray, alpha=alpha, beta=beta)
                                # apply GuassianBlur to reduce noise. medianBlur is also added for smoothening, reducing noise.
                                gray = cv2.GaussianBlur(gray_light,(11,11),0)
                                gray = cv2.medianBlur(gray, 19)
                                ##thresholded edge contouring
                                hist = cv2.equalizeHist(gray)
                                gamma = 2
                                invGamma = 1/gamma
                                table = np.array([((i / 255.0) ** invGamma) * 255
                                                  for i in np.arange(0, 256)]).astype("uint8")
                                gamm = cv2.LUT(hist, table, hist)
                                #submitted = cv2.Canny(gamm, 10, 50)            # why is minVal bigger than maxValue? original code was (gamm, 20, 10)
                                submitted = cv2.Canny(gamm, canny_min, canny_max)
                                submitted = cv2.GaussianBlur(submitted,(7,7),0)
                                submitted = cv2.threshold(submitted, 25, 255, cv2.THRESH_BINARY)[1]
                                submitted = cv2.dilate(submitted, None, iterations=9)  
                                #submitted = cv2.erode(submitted, None, iterations=10) 
                                submitted = cv2.erode(submitted, None, iterations=erode_it_1)
                                submitted = cv2.GaussianBlur(submitted,(7,7),0) 
                                #submitted = cv2.erode(submitted, None, iterations=4)
                                submitted = cv2.erode(submitted, None, iterations=erode_it_2)

                                #track demicircles (note original prototype values: dp = 1, minDist = 10000, param1=1, param2=10, minRadius = 5, maxRadius= 250)                              
                                circles = cv2.HoughCircles(submitted, cv2.HOUGH_GRADIENT, 
                                                           dp = dp,minDist = minDist,  
                                                           param1 = param1,param2 = param2, 
                                                           minRadius = minRadius, maxRadius = maxRadius)
                                #round the values
                                if circles is not None:
                                    circles = np.round(circles[0, 0:1]).astype("int")
                                #save it to a row
                                new_row = [i, circles[0,0], circles[0,1],circles [0,2], im, alpha, beta, canny_min, canny_max,erode_it_1,erode_it_2,param2]
                                df.loc[len(df)]=new_row
                                
                                if i in checkpoints:
                                    filename = '../TestResults/sens_analysis_all_comb_tracked_31082022_checkpoint'+ str(i) + '.csv'
                                    df.to_csv(filename, sep = ',')
                                    print("Checkpoint reached!")
                                    
                                
            
print('done')

#save final!
filename = '../TestResults/sens_analysis_all_comb_tracked_31082022_final' + '.csv'
df.to_csv(filename, sep = ',')


Checkpoint reached!
